# LLM Research Agent Quickstart

This notebook demonstrates an end-to-end LLM research workflow using mocked
services so it can run entirely offline. Follow the accompanying
:doc:`../llm_research_agent` guide for environment setup and security notes.

## 1. Configure the environment

Set the environment variables that the agent relies on. In production you would
load them from a secure store; here we use temporary in-memory values.

In [ ]:
import os
from pathlib import Path

os.environ['ACE_AGENT_DATA_DIR'] = '/tmp/ace-agent/data'
os.environ['ACE_AGENT_MODEL_CACHE'] = '/tmp/ace-agent/models'
os.environ['ACE_AGENT_API_KEY'] = 'mock-api-key'
os.environ['ACE_AGENT_RESULTS_BUCKET'] = 'file:///tmp/ace-agent/results'
os.environ['ACE_AGENT_LOG_LEVEL'] = 'INFO'

for key in sorted(k for k in os.environ if k.startswith('ACE_AGENT_')):
    print(f"{key}={os.environ[key]}")


## 2. Assemble a mocked dataset

Real experiments would read from ``ACE_AGENT_DATA_DIR``. The snippet below
creates a lightweight dataset for illustrative purposes.

In [ ]:
import json

DATASET_PATH = Path(os.environ['ACE_AGENT_DATA_DIR']) / 'prompts.json'
DATASET_PATH.parent.mkdir(parents=True, exist_ok=True)
mock_dataset = [
    {
        'id': 'sample-001',
        'prompt': 'Summarize the latest ocean temperature anomaly findings.',
        'context': 'Global SST anomalies remain above the 1985-2014 baseline.'
    },
    {
        'id': 'sample-002',
        'prompt': 'Describe possible drivers of the observed Arctic amplification.',
        'context': 'Sea-ice loss and increased poleward moisture transport are key.'
    },
]
DATASET_PATH.write_text(json.dumps(mock_dataset, indent=2))
print(f'Wrote {DATASET_PATH}')


## 3. Define a mocked LLM interface

The project typically integrates with remote providers. For testing we inject a
mock implementation that produces deterministic responses based on the prompt.

In [ ]:
from dataclasses import dataclass
from typing import List, Dict


@dataclass
class MockLLM:
    name: str = 'mock-llm-v1'

    def generate(self, prompt: str, context: str) -> str:
        return (
            f"[{self.name}] Prompt: {prompt}\n"
            f"Context: {context}\n"
            "Response: Further analysis scheduled."
        )


def run_agent(prompts: List[Dict[str, str]], llm: MockLLM):
    results = []
    for entry in prompts:
        response = llm.generate(entry['prompt'], entry['context'])
        results.append({
            'id': entry['id'],
            'response': response,
        })
    return results


mock_llm = MockLLM()
print(f'Initialized {mock_llm.name}')


## 4. Execute the research workflow

Load the dataset, generate outputs, and persist the findings to the results
directory specified in ``ACE_AGENT_RESULTS_BUCKET``.

In [ ]:
RESULTS_PATH = Path('/tmp/ace-agent/results')
RESULTS_PATH.mkdir(parents=True, exist_ok=True)

with DATASET_PATH.open() as f:
    prompts = json.load(f)

results = run_agent(prompts, mock_llm)
OUTPUT_FILE = RESULTS_PATH / 'mock_results.json'
OUTPUT_FILE.write_text(json.dumps(results, indent=2))
print(f'Saved {len(results)} results to {OUTPUT_FILE}')
print('
First response:
', results[0]['response'])


## 5. Next steps

* Swap ``MockLLM`` for your production client implementation.
* Replace the synthetic dataset with curated experiments stored in ``ACE_AGENT_DATA_DIR``.
* Integrate evaluation metrics or visualization notebooks to compare experiment runs.